Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# 04. Train in a remote VM (MLC managed DSVM)
* Create Workspace
* Create Project
* Create `train.py` file
* Create DSVM as Machine Learning Compute (MLC) resource
* Configure & execute a run in a conda environment in the default miniconda Docker container on DSVM

## Prerequisites
Make sure you go through the [00. Installation and Configuration](00.configuration.ipynb) Notebook first if you haven't.

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Create Experiment

In [ ]:
experiment_name = 'train-on-remote-vm'
script_folder = './samples/train-on-remote-vm'

import os
os.makedirs(script_folder, exist_ok = True)

In [ ]:
from azureml.core import Experiment

exp = Experiment(workspace = ws, name = experiment_name)

## Create `train.py`

Use `%%writefile` magic to write training code to `train.py` file under your project folder.

In [ ]:
%%writefile $script_folder/train.py

import os
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from azureml.core.run import Run
from sklearn.externals import joblib

import numpy as np

os.makedirs('./outputs', exist_ok=True)

X, y = load_diabetes(return_X_y = True)

run = Run.get_submitted_run()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
data = {"train": {"X": X_train, "y": y_train},
        "test": {"X": X_test, "y": y_test}}

# list of numbers from 0.0 to 1.0 with a 0.05 interval
alphas = np.arange(0.0, 1.0, 0.05)

for alpha in alphas:
    # Use Ridge algorithm to create a regression model
    reg = Ridge(alpha = alpha)
    reg.fit(data["train"]["X"], data["train"]["y"])

    preds = reg.predict(data["test"]["X"])
    mse = mean_squared_error(preds, data["test"]["y"])
    run.log('alpha', alpha)
    run.log('mse', mse)
    
    model_file_name = 'ridge_{0:.2f}.pkl'.format(alpha)
    with open(model_file_name, "wb") as file:
        joblib.dump(value = reg, filename = 'outputs/' + model_file_name)

    print('alpha is {0:.2f}, and mse is {1:0.2f}'.format(alpha, mse))

## Create Linux DSVM as a compute target

**Note**: If creation fails with a message about Marketplace purchase eligibilty, go to portal.azure.com, start creating DSVM there, and select "Want to create programmatically" to enable programmatic creation. Once you've enabled it, you can exit without actually creating VM.
 
**Note**: By default SSH runs on port 22 and you don't need to specify it. But if for security reasons you switch to a different port (such as 5022), you can append the port number to the address like the example below. [Read more](../../documentation/sdk/ssh-issue.md) on this.

In [ ]:
from azureml.core.compute import DsvmCompute
from azureml.core.compute_target import ComputeTargetException

compute_target_name = 'mydsvm'

try:
    dsvm_compute = DsvmCompute(workspace = ws, name = compute_target_name)
    print('found existing:', dsvm_compute.name)
except ComputeTargetException:
    print('creating new.')
    dsvm_config = DsvmCompute.provisioning_configuration(vm_size = "Standard_D2_v2")
    dsvm_compute = DsvmCompute.create(ws, name = compute_target_name, provisioning_configuration = dsvm_config)
    dsvm_compute.wait_for_completion(show_output = True)

## Attach an existing Linux DSVM as a compute target


In [ ]:
'''
 from azureml.core.compute import RemoteCompute 
 # if you want to connect using SSH key instead of username/password you can provide parameters private_key_file and private_key_passphrase 
 dsvm_compute = RemoteCompute.attach(ws,name="attach-from-sdk6",username=<username>,address=<ipaddress>,ssh_port=22,password=<password>)
'''

## Configure & Run

### Configure a Docker run with new conda environment on the VM
You can execute in a Docker container in the VM. If you choose this route, you don't need to install anything on the VM yourself. Azure ML execution service will take care of it for you.

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies


# Load the "cpu-dsvm.runconfig" file (created by the above attach operation) in memory
run_config = RunConfiguration(framework = "python")

# Set compute target to the Linux DSVM
run_config.target = compute_target_name

# Use Docker in the remote VM
run_config.environment.docker.enabled = True

# Use CPU base image from DockerHub
run_config.environment.docker.base_image = azureml.core.runconfig.DEFAULT_CPU_IMAGE
print('Base Docker image is:', run_config.environment.docker.base_image)

# Ask system to provision a new one based on the conda_dependencies.yml file
run_config.environment.python.user_managed_dependencies = False

# Prepare the Docker and conda environment automatically when executingfor the first time.
run_config.prepare_environment = True

# specify CondaDependencies obj
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

### Submit the Experiment
Submit script to run in the Docker image in the remote VM. If you run this for the first time, the system will download the base image, layer in packages specified in the `conda_dependencies.yml` file on top of the base image, create a container and then execute the script in the container.

In [ ]:
from azureml.core import Run
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory = script_folder, script = 'train.py', run_config = run_config)
run = exp.submit(src)
run.wait_for_completion(show_output = True)

### View run history details

In [ ]:
run

### Find the best run

In [ ]:
# get all metris logged in the run
run.get_metrics()
metrics = run.get_metrics()

In [ ]:
import numpy as np
print('When alpha is {1:0.2f}, we have min MSE {0:0.2f}.'.format(
    min(metrics['mse']), 
    metrics['alpha'][np.argmin(metrics['mse'])]
))

## Clean up compute resource

In [ ]:
dsvm_compute.delete()